In [ ]:
%matplotlib inline

# Non-GP regressor.


In [ ]:
from __future__ import annotations

from gemseo import configuration
from gemseo import sample_disciplines
from gemseo.mlearning.regression.algos.rbf import RBFRegressor

from gemseo_mlearning.active_learning.active_learning_algo import ActiveLearningAlgo
from gemseo_mlearning.active_learning.distributions.regressor_distribution import (
    RegressorDistribution,
)
from gemseo_mlearning.problems.rosenbrock.rosenbrock_discipline import (
    RosenbrockDiscipline,
)
from gemseo_mlearning.problems.rosenbrock.rosenbrock_space import RosenbrockSpace

# Update the configuration of |g| to speed up the script.
configuration.fast = True

The use of active learning methods
dedicated to exploration
is illustrated in this example,
with all default settings.
The function to approximate is
the Rosenbrock function $f(x_1,x_2)=(1-x_1)^2+100(x_2-x_1^2)^2$:



In [ ]:
discipline = RosenbrockDiscipline()

with $x_1$ and $x_2$ belonging to $[-2,2]^2$:



In [ ]:
input_space = RosenbrockSpace()

First,
we create an initial training dataset using an optimal LHS including 10 samples:



In [ ]:
learning_dataset = sample_disciplines(
    [discipline], input_space, "y", algo_name="OT_OPT_LHS", n_samples=10
)

and a universal regressor, namely a radial basis function network based on SciPy:



In [ ]:
regressor = RBFRegressor(learning_dataset)
regressor_distribution = RegressorDistribution(regressor, bootstrap=False, size=50)
regressor_distribution.learn()

Then,
we look for 20 points that will help us
to improve the overall accuracy
of the surrogate model.
By default,
for this purpose,
the active learning algorithm looks
for the point maximizing
the regressor local variance
with the help of the SLSQP gradient-based algorithm
applied in a multistart framework.



In [ ]:
active_learning = ActiveLearningAlgo("Exploration", input_space, regressor_distribution)
active_learning.acquire_new_points(discipline, 20)

Lastly,
we plot the training points,
the original model,
the RBF regressor
and the variance
after the last acquisition:



In [ ]:
active_learning.plot_acquisition_view(discipline=discipline)